In [1]:
using Plots, StatsBase, LinearAlgebra, Statistics, JLD2, Dates, StaticArrays, JSON

# data structure module
# include("spin_sim_functions_limited.jl")
# using .spin_sim_functions_limited

# function libraries
# include("spin_sims_new_limited.jl");

include("liouville_tools.jl")
using .LiouvilleTools

# SET VARIABLES

In [9]:
# make the parameter file
params = make_params()

# interaction
params["α"] = 0.1;

# number of frequencies
params["n"] = (50, 50)

# make a lattice, pbc = periodic bc or not
params["hlk"] = [1; 1]
params["θ"] = [π/2]
params["r"], params["spin_idx"] = make_lattice(params["hlk"], params["θ"], params["n"])

# make the stencil
params["ξ"] = 10
params["decay_power"] = 4
params["M_stencil"] = make_stencil(params["r"], params["ξ"], params["decay_power"])

# dissipation parameters
params["Γ"] = (0, 0, 10^-3);

# load the pulsing parameters
params["flip_angle"] = π/2
params["phases"] = (0, π/2)

# cpmg parameters
params["echo_time"] = 50e-6; # collect(LinRange(50e-6, 300e-6, 20))
params["num_echoes"] = convert.(Int64, round.(500e-6./params["echo_time"]));

# ASSIGN VARIABLES TO PARAMETER FILE

In [10]:
# set variables
vars = [];
params["vars"] = vars;

In [13]:
idx_list, d = make_idx(vars, params)
M_list = Array{Any}(undef, d)

for i in idx_list

    # generate temporary parameters
    tparams = make_temp_params(params, vars, i)

    # simulate
    # @time M_list[1] = spin_echo_sim_liouville(tparams);
    @time M_list[1], echoes = spin_echo_sim_liouville_cpmg(tparams);

end

#=
@save "cpmg_vary_α_0_vary_te_M_list.jld2" M_list
@save "cpmg_vary_α_0_vary_te_echoes.jld2" echoes
fname = "cpmg_vary_α_0_vary_te_params.json"
open(fname,"w") do f
    JSON.print(f, params)
end
=#

145.167970 seconds (170.81 M allocations: 246.569 GiB, 17.65% gc time)
